In [10]:
# -*- coding: utf-8 -*- 
import urllib , requests , sys
from bs4 import BeautifulSoup

#爬comiclist
#key=作者
key = '沢田大介'

res = requests.get("http://comiclist.jp/index.php?p=s&mode=ss&keyword=%E6%B2%A2%E7%94%B0%E5%A4%A7%E4%BB%8B&type=title")
#亂碼處理
res.encoding =  res.apparent_encoding
soup = BeautifulSoup(res.text)

a = 0
print soup.select('.search_page_link_info')[1].text
print soup.select('.search_page_link_info')[1]
'''
for "list-line" in soup.select('td[class^="list-line&nbsp;list-book"]'):
#for tr in soup.select('tr'):
    b = a * 18 + 3
    
    #print soup.select('.More Info')
    print soup.select('strong')
    #print soup.select('.thumbnail')[a]
'''

for strong in soup.select('strong'):
#while soup.select('strong')[a]:
#while soup.select('td[class^="list-line"]')[a]:
    print soup.select('strong')[a].text
    a = a + 1



#print soup.select('td[class^="list-line"]')
#print res.text



10 件中 1 － 10 のデータを表示します

<div class="search_page_link_info">
<font color="#dd0000"><b>10</b></font> 件中 <font color="#dd0000"><b>1</b></font> － <font color="#dd0000"><b>10</b></font> のデータを表示します<br/>
</div>
（成）絶対隷母
（成）噂の奥さん、ド変態！　－人妻・響子とシませんか？－
（成）天然交母
（成）羞恥肉林
（成）痴女～派
（成）ママごと
クリーム
（成）ＤＥＥＰ
（成）母淫妻（ボインヅマ）
（成）ＳＨＡＫＥ
